<h1> Similarity Search - Embeddings </h1>
<br>

Before starting, please make sure this notebook is using **conda_python3** kernel from the top right!

Run all the cells and inspect the output of each cell.

### What are embeddings?

Embedding refers to the process of transforming objects such as text, images, video, or audio into numerical representations that reside in a high-dimensional vector space. In short, embeddings represent items like words as vectors of numbers. After generating these embeddings, an application can perform similarity searches within the vector space. Words with similar meanings will have similar vectors.

The following diagram provides a visual representation of what this looks like for word embeddings i.e., words that are semantically similar are close together in the embedding space. 

![Vector](../images/vector.jpeg)

### Introduction

In this notebook, you will create a **Similarity Search** application using Amazon Titan Embeddings model and Amazon RDS. We use vector embeddings to retrieve similar products in the catalog based on our search keyword. 


* You will search products in the catalog using a search keyword. Example: red dress for a wedding.
* The search keyword will be converted into vector embeddings at runtime using Titan Embeddings  model.
* These search embeddings will be queried against the Product embeddings we stored in the RDS database using pgvector extension. This query will retrieve products similar to the search keyword.
* The similarity search results will be displayed.

![Vector Search](../images/vector-search.png)

### Install required dependencies

**Important:** You may see an ERROR or a warning that "you may need to restart the kernel" from the following cell. **Ignore** and proceed with the next cells. 

In [ ]:
%pip install --quiet --no-build-isolation --upgrade \
    "boto3==1.28.63" \
    "awscli==1.29.63" \
    "botocore==1.31.63" \
    "langchain==0.0.309" \
    "psycopg2-binary==2.9.9" \
    "pgvector==0.2.3" \
    "numpy==1.26.1"

<h3> Import required packages </h3>

In [ ]:
import json
import os
import sys
import boto3
import botocore
from langchain import PromptTemplate
from langchain.llms.bedrock import Bedrock

# For vector search
from langchain.embeddings import BedrockEmbeddings
import psycopg2
from pgvector.psycopg2 import register_vector
import numpy as np

# For image operations
from PIL import Image
import base64
import io
import requests

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww

<h3> Initialize Bedrock client </h3>

In [ ]:
boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None)
)

#### Initialize Amazon Bedrock Embeddings model 

We are using Titan Embeddings model to convert our search keyword to vector embeddings to perform similarity search. 

In [ ]:
modelId = "amazon.titan-embed-text-v1"
bedrock_embeddings = BedrockEmbeddings(model_id=modelId, client=boto3_bedrock)

#### Define a search keyword and create vector embedding for that keyword

In [ ]:
keyword = "floral prints"
print(keyword)

#### Now let's create vector embedding for this keyword using Bedrock

In [ ]:
search_embedding = list(bedrock_embeddings.embed_query(keyword))
print(search_embedding)

We are going to use the search embeddings to query the RDS vector knowledge base. This vector database is already prepopulated with embeddings for all the products in [this](https://github.com/zalandoresearch/feidegger/blob/master/data/FEIDEGGER_release_1.2.json) catalog. We used the same [FEIDEGGER](https://github.com/zalandoresearch/feidegger/tree/master) dataset to generate all the vector embeddings. 

Please note that in order to save time, embeddings for all the 8732 products in this dataset have been **pre-populated** into your Amazon RDS database. The process to create vector embeddings for these many embeddings takes about ~20-30 minutes. In order to store and query these embeddings, your RDS database needs to have [pgvector](https://github.com/pgvector/pgvector) extension installed. It has also been pre-installed in your RDS instance.

This is the [notebook](https://github.com/aws-samples/retails-generative-ai-workshop/blob/ai/notebooks/create_embeddings_with_titan.ipynb) that shows how we pre-populated the embeddings. This is **just for your reference**. We recommend reviewing this code and other documentations **after the workshop**.

#### Now lets connect to Amazon RDS and query the pre-populated embeddings based on the search keyword 

In [ ]:
# Initialize secrets manager
secrets = boto3.client('secretsmanager')

sm_response = secrets.get_secret_value(SecretId='postgresdb-secrets')

database_secrets = json.loads(sm_response['SecretString'])

dbhost = database_secrets['host']
dbport = database_secrets['port']
dbuser = database_secrets['username']
dbpass = database_secrets['password']
dbname = database_secrets['vectorDbIdentifier']

# Connect to the RDS vectordb database 
dbconn = psycopg2.connect(host=dbhost, user=dbuser, password=dbpass, port=dbport, database=dbname, connect_timeout=10)
dbconn.set_session(autocommit=True)
register_vector(dbconn)
cur = dbconn.cursor()

#### Execute search query 

Here, we perform similarity search by querying the search keyword embedding against the pre-populated product embeddings.

In [ ]:
# Limiting search result to 2 for now
cur.execute("""SELECT id, url, description, descriptions_embeddings 
                        FROM vector_products
                        ORDER BY descriptions_embeddings <-> %s limit 2;""", 
                        (np.array(search_embedding),))

# Fetch search result
dbresult = cur.fetchall()

#### Display similarity search result

This search result contains top 2 products that are similar to our search keyword. 

In [ ]:
for x in dbresult:
    # Get similar product IDs
    product_item_id = x[0]
    
    # Get similar product descriptions
    desc = x[2]
    
    # Get image from URL
    url = x[1].split('?')[0]
    response = requests.get(url)
    img = Image.open(io.BytesIO(response.content))
    img = img.resize((256, 256))
    
    # Print similarity search results
    print("Product ID: " +str(product_item_id))
    print("\n"+desc)
    img.show()


### You've successfully created a similarity search application using Amazon Bedrock Embeddings!

#### Now, let's integrate this feature into our retail web application. Please go back to Workshop Studio and follow the instructions to build this feature using your Cloud9 IDE.